# Outbound Auth

Outbound Auth allows agents and the AgentCore Gateway to securely access AWS resources and third-party services on behalf of users who have been authenticated and authorized during Inbound Auth. To integrate authorization with an AWS resource or third-party service, it's necessary to configure both Inbound Auth and Outbound Auth.

With just-enough access and secure permission delegation supported by AgentCore Identity, agents can seamlessly and securely access AWS resources and third-party tools such as GitHub, Google, Salesforce, and Slack. Agents can perform actions on these services either on behalf of users or independently, provided there is pre-authorized user consent. Additionally, you can reduce consent fatigue using a secure token vault and create streamlined AI agent experiences.

## Outbound Authentication Configuration

First, you register your client application with third-party providers and then create an Outbound Auth. You specify how you want to validate access to the AWS resource or third-party service or AgentCore Gateway targets. You can use OAuth 2LO/3LO or API keys. With OAuth, you can select from providers that AgentCore Identity provides. In which case you enter the configuration details for the providers from AgentCore Identity. Alternatively, you can supply details for a custom provider. 

When a user wants access to an AWS resource or third-party service or AgentCore Gateway target, the Outbound Auth confirms that the access tokens provided by Incoming Auth are valid and if so, allows access to the resource.

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>


Here are the various parameters you can use with the @require_access_token decorator.


| Parameter Name      | Description                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | The credential provider name                                             |
| into                | Parameter name to inject the token into                                  |
| scopes              | OAuth2 scopes to request                                                 |
| on_auth_url	      | Callback for handling authorization URLs                                 |
| auth_flow           | Authentication flow type ("M2M" or "USER_FEDERATION")                    |
| callback_url        | OAuth2 callback URL                                                      |
| force_authentication| Force re-authentication                                                  |
| token_poller        | Custom token poller implementation                                       |

		


# Hosting Strands Agents in Amazon Bedrock AgentCore Runtime

## Overview


In this tutorial we will develop a scheduling agent using Strands agents that can list the events from the users Google Calendar. We will configure a credential provider to help with credential management with Google. We can use the named provider for Google and modify our agent code to call the credential provider and use the access_token to get the users calendar events or schedule from Google.

### Tutorial Architecture

<div style="text-align:center">
    <img src="images/outbound_auth_3lo.png" width="90%"/>
</div>


### Tutorial Details

| Information         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                           |
| Agent type          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | Anthropic Claude Haiku 4.5                                              |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Claude Model |
| Tutorial vertical   | Cross-vertical                                                           |
| Example complexity  | Medium                                                                   |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                             |
| Credential Provider | Type : OAuth2 - Google Provider                                          |


### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Claude models
* Using Strands Agents
* Using AgentCore egress Auth with OAuth2 Google credential provider.


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
sagemaker-studio 1.1.1 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.9 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.2.8 requires numpy<=2.0.1, but you have numpy 2.3.5 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.3.5 which is incompatible.
autogluon-common 1.4.0 requires psuti

##  Configure Inbound Auth with Cognito as the IDP
Lets provision a Cognito Userpool with an App client and one test user. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. To do so, we will use the `setup_cognito_user_pool` support function from our `utils` script.

Note: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.

In [2]:
import sys
import os
import subprocess
from boto3.session import Session

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])
from utils import setup_cognito_user_pool, reauthenticate_user

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

sys.path[0]: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples
Region: us-west-2


In [3]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool("Cognito_3LO_Google")
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-west-2_Ig2LnJ6dS
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_Ig2LnJ6dS/.well-known/openid-configuration
Client ID: 3f3lm5eg2scuq7c4lcc84fjejm
Bearer Token: eyJraWQiOiJPaDdOdllVXC9LUWtkczBOVXg2V1dMdVdmS0U5N3BcL1BSMXZkejJkUVZTWE09IiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiI1ODgxMjMzMC01MGYxLTcwNjItODgzMS05MTNhMmJhOGZjMDkiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9JZzJMbko2ZFMiLCJjbGllbnRfaWQiOiIzZjNsbTVlZzJzY3VxN2M0bGNjODRmamVqbSIsIm9yaWdpbl9qdGkiOiIzZTEyNGFkOC1hYzc4LTRjMDgtODJjOC0yOTYyMjcwZWNhYTMiLCJldmVudF9pZCI6ImQ1OWQxYjA3LTVkYjctNGY5OS1iZjZkLWY1MWQxOGU5MWU4YiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjU2NDE1NTIsImV4cCI6MTc2NTY0NTE1MiwiaWF0IjoxNzY1NjQxNTUyLCJqdGkiOiI2NjY1MjUwMS05YjM3LTQ4YjItYmE0OS01YWVmMGIxNTI5NTgiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.lYW8d9xjoyewL12PeEf2h9PkocMmnnUZpWIzAoRF0AfLbyRoR6MC_kRqxkk4GoToJPRi6Qejx

## Configure Google for OAuth2. ( On behalf of user flow )
Follow these steps to register your app, create a project, and configure OAuth credentials for Google Calendar access:

In this section, we will configure your Google to access the Google Calendar API with readonly permissions.

1. Create a Project in Google Developer Console
    1.    Go to the [Google Developer Console](https://console.developers.google.com/)
    2.    In the top navigation bar, click on “Create Project”
    3.    Enter a Project Name.
    4.    Choose an Organization or leave as “No organization” if not applicable.
    5.    Click Create. Your new project will appear in the project list.
2. Enable Google Calendar API
    1.    With your project selected ( using the checkbox),  open the left menu (hamburger menu) and go to APIs & Services > Library.
    2.    In the search bar, type Google Calendar API.
    3.    Click on Google Calendar API in the results, then click Enable.
3. Configure OAuth Consent Screen
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Click “Get started”
    3.    Fill in the required fields:
    4.    App Name
    5.    User Support Email
    6. Click Next and then select the right Audience type, i.e. Internal or External, Click Next, ( If you select External, then ensure you have entered email ids of users entered here, so you can test with those users)
    7.    Developer Contact Information, Enter your email id.
    8.  Click “Finish” after accepting the terms and conditions. an then click “Create”.
4. Add your google email id as a test user
    1.    In the left menu, go to APIs & Services > OAuth consent screen.
    2.    Select "Audience" from the left hand menu.
    3.    Under Test users, click "+ Add Users" and add your gmail id for the Google account.
5. Create OAuth 2.0 Credentials
    1.    Go to APIs & Services from the left hand menu and then select  > Credentials.
    2.    Click Create Credentials > OAuth client ID.
    3.    Select Web application as the application type.
    4.    Enter a name for the credentials.
    5.    Click Create.
6. Obtain Client ID and Client Secret
    1.    After creation, a dialog will display your Client ID and Client Secret. Copy these for later use.
    2.    Download the credentials as a JSON file or copy them for use in your app’s configuration.
7. Update the Data access 
    1. Go to APIs & Services from the left hand menu and then select  > Credentials.
    2. Select the “web app” you created in step 4d.
    3. Select “Data access” on the left hand menu
    4. Click “Add or remove scopes”
    5. Add scopes based on your use case. E.g: For Google calendar add, “https://www.googleapis.com/auth/calendar.readonly” under “Manually add scopes” and click update followed by Save/Update.
    6. Click “Save” again on the “Data access” page to save your configuration.
8. Use the Credentials in Your Agent
    1.    In the next section, we will configure a resource credentials provider to use the Client ID, Client Secret, and Redirect URI for the OAuth 3-legged flow.

## OAuth2 Authorization URL Session Binding Process

The OAuth2 authorization URL session binding process is a critical security mechanism that ensures OAuth2 authorization sessions are properly associated with authenticated users in AgentCore Identity. This process prevents session hijacking and ensures that OAuth tokens are only granted to the intended user.

Ref : https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html

### How session binding works
<div style="text-align:center">
    <img src="images/identity-session-binding.png" width="90%"/>
</div>

1. Invoke agent – Your agent code invokes GetResourceOauth2Token API to retrieve an authorization URL, when an originating agent user wants to access some application or resource that he/she owns.

2. Generate authorization URL – AgentCore Identity generates an authorization URL and session URI for the user to navigate to and consent access.

3. Authorize and obtain access token – The user navigates to the authorization URL and grants consent for your agent to access his/her resource. After that, AgentCore Identity redirects the user's browser to your HTTPS application endpoint with information containing the originating user of the authorization request. At this point, your HTTPS application endpoint determines if the originating agent user is still the same as the currently logged in user of your application. If they match, your application endpoint invokes CompleteResourceTokenAuth so that AgentCore Identity can fetch and store the access token.

4. Re-invoke agent to obtain access token – Once the application returns a valid response, your agent application will be able to retrieve the OAuth2.0 access tokens that were originally requested for the user. If the users do not match, your application simply does nothing or logs the attempt.

By allowing your application endpoint to verify the user identity, AgentCore Identity allows your agent application to ensure that it is always the same user who initiated the authorization request and the one who consented access.

### Overview of the OAuth2 Session Binding Flow in this sample

The OAuth2 session binding process involves several key steps that coordinate between your application, AgentCore Identity, external OAuth providers (like Google, Github ), and a local callback server:

#### Step 1: Create Application Callback URL
- Create a publicly accessible HTTPS callback endpoint in your application
- This endpoint will handle OAuth redirects and validate user sessions
- Example: `https://myagentapp.com/callback`

#### Step 2: Update Workload Identity with Callback URL
- Register your callback URL as an `AllowedResourceOauth2ReturnUrl` in the workload identity
- This is accomplished using the `UpdateWorkloadIdentity` API
- **In this tutorial**: This step is handled automatically by the code below that updates the workload identity with the local callback server URL

#### Step 3: Create OAuth2 Credential Provider
- Configure the credential provider with external OAuth provider details (client ID, secret)
- AgentCore Identity returns a unique callback URL for the provider
- Register this callback URL with the external OAuth provider (e.g., Google Console)

#### Step 4: Implement Session Validation and Token Completion
- Your callback endpoint must validate the current user's session
- Call `CompleteResourceTokenAuth` API with the user identifier and session URI
- **In this tutorial**: The `oauth2_callback_server.py` handles this automatically

#### Step 5: OAuth Flow Execution
- User triggers OAuth flow through agent interaction
- User is redirected to external provider for authorization
- Provider redirects back to your callback with session information
- Session binding completes and OAuth tokens become available

### Local Development with oauth2_callback_server.py

For local development and testing, this tutorial uses `oauth2_callback_server.py` which:

1. **Runs a Local FastAPI Server** (`localhost:9090`)
   - Provides `/ping` endpoint for health checks
   - Provides `/userIdentifier/token` endpoint to store user tokens
   - Provides `/oauth2/callback` endpoint to handle OAuth redirects

2. **Manages User Token Storage**
   - Stores the user's JWT token from Cognito authentication
   - Associates OAuth sessions with the correct user identity

3. **Handles OAuth Callback Processing**
   - Receives OAuth redirects with `session_id` parameter
   - Calls `CompleteResourceTokenAuth` to bind the session
   - Validates user identity before completing the flow

4. **Provides Session Security**
   - Ensures OAuth sessions are bound to authenticated users
   - Prevents unauthorized access to OAuth tokens

### Integration with Workload Identity Update

The code snippet you referenced performs a crucial step in the OAuth2 session binding process:

```python
if launch_result.agent_id:
    workload_name = launch_result.agent_id
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
```

This code:
1. **Retrieves the Agent's Workload Identity**: Uses the agent ID from the runtime deployment
2. **Gets Current Allowed URLs**: Fetches existing `allowedResourceOauth2ReturnUrls`
3. **Adds Local Callback URL**: Includes `http://localhost:9090/oauth2/callback` as an allowed return URL
4. **Updates Workload Identity**: Registers the callback URL with AgentCore Identity

This registration is essential because AgentCore Identity will only redirect OAuth callbacks to pre-registered URLs, providing an additional security layer.

### Security Considerations

The OAuth2 session binding process includes several security measures:
- **URL Validation**: Only pre-registered callback URLs are allowed
- **Session Verification**: User sessions must be validated before token completion
- **User Identity Binding**: OAuth sessions are explicitly bound to authenticated users
- **Token Isolation**: Each user's OAuth tokens are isolated and secure

This comprehensive approach ensures that OAuth2 flows are secure and properly attributed to the correct users in multi-user environments.

---

### Configure the Google OAuth2 credential provider.

Modify the following code and replace the following :
1. <your-google-client-id> with the "client id" recorded from Step 6.1 above
2. <your-google-client-secret> with the "client secret" recorded from Step 6.2 above.

Important : Please ensure you replace the entire place holder string including the "<" and ">"


Once the client id and client secret are updated, run the below code to create a credentials provider for Google. <br>
Resource credential providers in AgentCore Identity act as intelligent intermediaries that manage the complex relationships between agents, identity providers, and resource servers. Each provider encapsulates the specific endpoint configuration required for a particular service or identity system. The service provides built-in providers for popular services including Google, GitHub, Slack, and Salesforce, with authorization server endpoints and provider-specific parameters pre-configured to reduce development effort. AgentCore Identity supports custom configurations through configurable OAuth2 credential providers that can be tailored to work with any OAuth2-compatible resource server.



In [7]:
from bedrock_agentcore.services.identity import IdentityClient

identity_client = IdentityClient(region)

# Configure Google OAuth2 provider - On-Behalf-Of User
google_provider = identity_client.create_oauth2_credential_provider({
    "name": "google-cal1-provider",
    "credentialProviderVendor": "GoogleOauth2",
    "oauth2ProviderConfigInput": {
        "googleOauth2ProviderConfig": {
            "clientId": "",
            "clientSecret": ""
        }
    }
})
print(google_provider)
print("\n")
print(f"callbackUrl: {google_provider['callbackUrl']}")

{'ResponseMetadata': {'RequestId': 'a221c8d6-19dd-4466-a0fb-965dcebb493e', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Sat, 13 Dec 2025 16:10:31 GMT', 'content-type': 'application/json', 'content-length': '704', 'connection': 'keep-alive', 'x-amzn-requestid': 'a221c8d6-19dd-4466-a0fb-965dcebb493e', 'x-amzn-remapped-x-amzn-requestid': '67e5a5df-f997-4acc-b610-d56ca513b390', 'x-amzn-remapped-content-length': '704', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'ViNuwG05vHcEqTg=', 'x-amzn-trace-id': 'Root=1-693d8ff7-61ed5a62149a4d533f9d11eb', 'x-amzn-remapped-date': 'Sat, 13 Dec 2025 16:10:31 GMT'}, 'RetryAttempts': 0}, 'clientSecretArn': {'secretArn': 'arn:aws:secretsmanager:us-west-2:455933813601:secret:bedrock-agentcore-identity!default/oauth2/google-cal1-provider-uj3hrG'}, 'name': 'google-cal1-provider', 'credentialProviderArn': 'arn:aws:bedrock-agentcore:us-west-2:455933813601:token-vault/default/oauth2credentialprovider/google-cal1-provider', 'callbackUrl': 'http

## Update the callback url on the Google/OAuth 2.0 client. 
Navigate back to the [Google Developer Console](https://console.developers.google.com/)

1. Select the Project in Google Developer Console
    1.    Select the project created earlier.
2. Update the callback uri.
    1.    Go to APIs & Services from the left hand menu and then select  > Credentials.
    2.    Click the client under "OAuth 2.0 Client IDs"
    3.    Under "Authorised redirect URIs", enter the callback url from the previous step. The callback url was printed so you can easily copy it form the previous step.
    4.    Click Save.

## Preparing your agent for deployment on AgentCore Runtime

### Strands Agent with a model hosted on Amazon Bedrock
Here is a Strands agent code that includes the following :
1. Creates a new tool called "get_calendar_events_today", to get the events from your Google Calendar for today
2. Uses the Credential provider created in the previous step to fetch the access_token from Google. This includes the user consent flow where the consent is sent to the user for approval as part of the 3LO flow.
3. The Strands agent calls the tool for any user requests related to the users agenda.

In [8]:
# Get the OAuth2 callback URL based on the current environment (notebook/SageMaker)
# This is evaluated HERE in the notebook, not in the agent container
from oauth2_callback_server import get_oauth2_callback_url
oauth2_callback_url_for_agent = get_oauth2_callback_url()

print(f"Callback URL for agent (determined from notebook environment): {oauth2_callback_url_for_agent}")

# Write strands_claude_google_3lo.py with the callback URL embedded as a string literal
google_agent_code = f'''import os
import datetime
import json
import asyncio
import traceback
from typing import Dict, Any, Optional, AsyncGenerator

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Environment configuration
os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

# Required OAuth2 scope for Google Calendar API
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

# Global variable to store the access token
google_access_token: Optional[str] = None


@tool(
    name="Get_calendar_events_today",
    description="Retrieves the calendar events for the day from your Google Calendar"
)
def get_calendar_events_today() -> str:
    """
    Retrieve calendar events for today from Google Calendar.
    
    Returns:
        str: JSON string containing events or error information
    """
    global google_access_token
    
    # Check if we already have a token
    if not google_access_token:
        return json.dumps({{
            "auth_required": True,
            "message": "Google Calendar authentication is required. Please wait while we set up the authorization.",
            "events": []
        }})
    
    # Create credentials from the provided access token
    creds = Credentials(token=google_access_token, scopes=SCOPES)
    try:
        service = build("calendar", "v3", credentials=creds)
        
        # Calculate today's time range
        today_start = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        today_end = today_start.replace(hour=23, minute=59, second=59)
        
        # Format with CDT timezone (-05:00)
        time_min = today_start.strftime('%Y-%m-%dT00:00:00-05:00')
        time_max = today_end.strftime('%Y-%m-%dT23:59:59-05:00')
        
        events_result = (
            service.events()
            .list(
                calendarId="primary",
                timeMin=time_min,
                timeMax=time_max,
                singleEvents=True,
                orderBy="startTime",
            )
            .execute()
        )
        events = events_result.get("items", [])

        if not events:
            return json.dumps({{"events": []}})

        return json.dumps({{"events": events}})
        
    except HttpError as error:
        error_message = str(error)
        return json.dumps({{"error": error_message, "events": []}})
    except Exception as e:
        error_message = str(e)
        return json.dumps({{"error": error_message, "events": []}})


# Initialize the agent with tools and your preferred model choice
agent = Agent(
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[get_calendar_events_today]
)

# Initialize app and streaming queue
app = BedrockAgentCoreApp()


class StreamingQueue:
    """A queue for streaming responses asynchronously."""
    
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item: str) -> None:
        """Add an item to the queue."""
        await self.queue.put(item)

    async def finish(self) -> None:
        """Mark the queue as finished."""
        self.finished = True
        await self.queue.put(None)

    async def stream(self) -> AsyncGenerator[str, None]:
        """Stream items from the queue."""
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item


queue = StreamingQueue()


async def on_auth_url(url: str) -> None:
    """Handle authorization URL callback."""
    print(f"Authorization url: {{url}}")
    await queue.put(f"Authorization url: {{url}}")


async def agent_task(user_message: str) -> None:
    """
    Execute the agent task with authentication handling.
    
    Args:
        user_message: The user's input message
    """
    try:
        await queue.put("Begin agent execution")
        
        # Call the agent first to see if it needs authentication
        response = agent(user_message)
        
        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)
        
        # Check if the response indicates authentication is required
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth", 
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)
       
        if needs_auth:
            await queue.put("Authentication required for Google Calendar access. Starting authorization flow...")
            
            # Trigger the 3LO authentication flow
            try:
                global google_access_token
                google_access_token = await need_token_3lo_async(access_token='')
                await queue.put("Authentication successful! Retrying calendar request...")
                
                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"auth_error: {{auth_error}}")
                await queue.put(f"Authentication failed: {{str(auth_error)}}")
        
        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {{str(e)}}")
    finally:
        await queue.finish()


@requires_access_token(
    provider_name="google-cal-provider",
    scopes=SCOPES,
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
    callback_url="{oauth2_callback_url_for_agent}"  # ← Callback URL determined from notebook environment
)
async def need_token_3lo_async(*, access_token: str) -> str:
    """
    Handle 3-legged OAuth token retrieval.
    
    Args:
        access_token: The OAuth access token
        
    Returns:
        str: The access token
    """
    global google_access_token
    google_access_token = access_token
    return access_token


@app.entrypoint
async def agent_invocation(payload: Dict[str, Any]) -> AsyncGenerator[str, None]:
    """
    Main entrypoint for agent invocations.
    
    Args:
        payload: The request payload containing the prompt
        
    Yields:
        str: Streaming response items
    """
    user_message = payload.get(
        "prompt", 
        "No prompt found in input, please guide customer to create a json payload with prompt key"
    )
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))
    
    # Return the stream, but ensure the task runs concurrently
    async def stream_with_task() -> AsyncGenerator[str, None]:
        # Stream results as they come
        async for item in queue.stream():
            yield item
        
        # Ensure the task completes
        await task
    
    return stream_with_task()


if __name__ == "__main__":
    app.run()
'''

# Write the file
with open("strands_claude_google_3lo.py", "w") as f:
    f.write(google_agent_code)

print("✅ strands_claude_google_3lo.py written successfully with embedded callback URL")

Callback URL for agent (determined from notebook environment): https://2tsypbyep8wxkc3.studio.us-west-2.sagemaker.aws/jupyterlab/default/proxy/9090/oauth2/callback
✅ strands_claude_google_3lo.py written successfully with embedded callback URL


## Deploying the agent to AgentCore Runtime
The CreateAgentRuntime operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent.

Note: Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

Note : The authorizer_configuration is configured for Inbound Auth with Cognito.

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [9]:
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = Session()
region = boto_session.region_name
print(f"Region: {region}")

discovery_url = cognito_config.get("discovery_url")
client_id = cognito_config.get("client_id")
agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_google_3lo.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    memory_mode="NO_MEMORY",
    agent_name="strands_agent_google_3lo",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
print(response)

Entrypoint parsed: file=/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/strands_claude_google_3lo.py, bedrock_agentcore_name=strands_claude_google_3lo
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: strands_agent_google_3lo
Memory disabled
Network mode: PUBLIC


Region: us-west-2


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/.dockerignore
Setting 'strands_agent_google_3lo' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/.bedrock_agentcore.yaml


config_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/.dockerignore') runtime='Docker' runtime_type=None region='us-west-2' account_id='455933813601' execution_role=None ecr_repository=None auto_create_ecr=True s3_path=None auto_create_s3=False memory_id=None network_mode='PUBLIC' network_subnets=None network_security_groups=None network_vpc_id=None


## Review the AgentCore configuration

In [10]:
!cat .bedrock_agentcore.yaml

default_agent: strands_agent_google_3lo
agents:
  strands_agent_google_3lo:
    name: strands_agent_google_3lo
    entrypoint: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/05-Outbound_Auth_3lo/strands_claude_google_3lo.py
    deployment_type: container
    runtime_type: null
    platform: linux/arm64
    container_runtime: docker
    source_path: null
    aws:
      execution_role: null
      execution_role_auto_create: true
      account: '455933813601'
      region: us-west-2
      ecr_repository: null
      ecr_auto_create: true
      s3_path: null
      s3_auto_create: false
      network_configuration:
        network_mode: PUBLIC
        network_mode_config: null
      protocol_configuration:
        server_protocol: HTTP
      observability:
        enabled: true
      lifecycle_configuration:
        idle_runtime_session_timeout: null
        max_lifetime: null
    bedrock_agentcore:
      agent_id: null
      agent_arn: null

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [11]:
from oauth2_callback_server import get_oauth2_callback_url

# Deploy the agent to AgentCore Runtime and get deployment details
launch_result = agentcore_runtime.launch()
print(launch_result)

if launch_result.agent_id:
    # Extract the workload name from the deployed agent's ID for identity management
    workload_name = launch_result.agent_id
    # Retrieve the current workload identity configuration from AgentCore Identity
    workload_identity = identity_client.get_workload_identity(name=workload_name)
    # Extract existing OAuth2 callback URLs that are already registered for this workload
    allowed_resource_oauth_2_return_urls = workload_identity.get("allowedResourceOauth2ReturnUrls") or []
    # Get the local OAuth2 callback server URL for session binding (localhost:9090/oauth2/callback)
    oauth2_callback_url = get_oauth2_callback_url()
    print(f"Updating workload {workload_name} with callback url {oauth2_callback_url}")

    # Register the local callback URL with the workload identity to enable OAuth2 session binding
    updated_workload_identity = identity_client.update_workload_identity(
        name=workload_name,
        allowed_resource_oauth_2_return_urls=[*allowed_resource_oauth_2_return_urls, oauth2_callback_url],
    )
    print(updated_workload_identity)

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'strands_agent_google_3lo' to account 455933813601 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_agent_google_3lo
ECR repository available: 455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_agent_google_3lo
Getting or creating execution role for agent: strands_agent_google_3lo
Using AWS region: us-west-2, account ID: 455933813601
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_agent_google_3lo


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b
Starting execution role creation process for agent: strands_agent_google_3lo
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b
✓ Role created: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_agent_google_3lo
Role creation complete and ready for use with Bedrock AgentCore
Execution role available: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-b868d44c0b
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_agent_google_3lo
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-b868d44c0b
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-b868d44c0b
Cr

mode='codebuild' tag='bedrock_agentcore-strands_agent_google_3lo:latest' env_vars=None port=None runtime=None ecr_uri='455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_agent_google_3lo' agent_id='strands_agent_google_3lo-IiIa6Y67ij' agent_arn='arn:aws:bedrock-agentcore:us-west-2:455933813601:runtime/strands_agent_google_3lo-IiIa6Y67ij' codebuild_id='bedrock-agentcore-strands_agent_google_3lo-builder:98ecf46c-8b5f-46d8-9c15-4ba0ffd4d122' build_output=None
Updating workload strands_agent_google_3lo-IiIa6Y67ij with callback url https://2tsypbyep8wxkc3.studio.us-west-2.sagemaker.aws/jupyterlab/default/proxy/9090/oauth2/callback
{'ResponseMetadata': {'RequestId': 'c57c0bf2-ea58-4d43-a4e7-910bf5932ab0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 13 Dec 2025 16:14:23 GMT', 'content-type': 'application/json', 'content-length': '414', 'connection': 'keep-alive', 'x-amzn-requestid': 'c57c0bf2-ea58-4d43-a4e7-910bf5932ab0'}, 'RetryAttempts': 0}, 'name': 'strands_age

#### Add extra required policies to auto-created role

If you are using this on a new account where model has not been accessed before, you will need to add extra required policies to the auto-created role to allow the agent to access the model.

In [12]:
import json
import boto3
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "BedrockModelAccess",
            "Effect": "Allow",
            "Action": [
                "aws-marketplace:ViewSubscriptions",
                "aws-marketplace:Subscribe"
            ],
            "Resource": "*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [13]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: strands_agent_google_3lo


Final status: READY


### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

You will notice that the agent calls the "Get_calendar_events_today" tool and triggers the 3 Legged Outh flow. You will be presented with the Authorization Url. Click on the Authorization url Or copy/paste it in a new browser session/tab to complete the user consent flow.
Once the Authorization completes, The credential provider "google-cal-provider" will fetch the access_token from Google and complete the tool execution to fetch the events from your calendar.

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [14]:
from oauth2_callback_server import store_token_in_oauth2_callback_server, wait_for_oauth2_server_to_be_ready

bearer_token = reauthenticate_user(cognito_config.get("client_id"))

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    # Start the OAuth2 callback server
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print("Failed to start OAuth2 callback server to handle session binding "
              "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)")
    else:
        store_token_in_oauth2_callback_server(bearer_token)
        invoke_response = agentcore_runtime.invoke(
            {"prompt": "What is in my agenda for today? Highlight the main events!"},
            bearer_token=bearer_token
        )
        print(invoke_response)
finally:
    oauth2_callback_server_process.terminate()

INFO:     Started server process [44946]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:9090 (Press CTRL+C to quit)
Using JWT authentication


INFO:     127.0.0.1:34660 - "GET /ping HTTP/1.1" 200 OK
INFO:     127.0.0.1:34676 - "POST /userIdentifier/token HTTP/1.1" 200 OK


Begin agent execution

Authentication required for Google Calendar access. Starting authorization flow...

Authorization url: 
https://bedrock-agentcore.us-west-2.amazonaws.com/identities/oauth2/authorize?request_uri=urn%3Aietf%3Aparams%3Aoau
th%3Arequest_uri%3AMDU2NDJjZTEtMjcxYi00OTc3LTk3ZTYtZDFlNjFmYmZlZTU5

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [44946]


KeyboardInterrupt: 

In [ ]:
ACCESS BLOCKED

## Optional - Test the agent using a Streamlit App

You can test your deployed agent using a Streamlit web application that provides a nice chat interface. The `chatbot_app_cognito.py` file in this directory creates a web-based chatbot that:

- Automatically reads configuration from `.bedrock_agentcore.yaml`
- Provides Cognito authentication
- Shows a modern chat interface with streaming responses
- Handles the 3LO OAuth flow for Google Calendar access

### Running the Streamlit App

You can run the Streamlit app in several ways:

#### Option 1: Run from Jupyter Notebook (Current Directory)
- Run the cell below to start the Streamlit app directly from this notebook:
- Login: Use the credentials testuser / MyPassword123! (the default test user created by the Cognito setup)
- Test some simple prompts like "Tell me a joke"
- Test with a prompt that will trigger the `Get_calendar_events_today` tool like "What is on my agenda for today?".
- You will see the Authorization url returned. Click on the url or copy/paste the url to a new browser tab/window to complete the user consent flow.

In [ ]:
from chatbot_app_cognito import get_streamlit_url

# Change to the current directory where the chatbot_app_cognito.py file is located
notebook_dir = os.getcwd()

# Start the Streamlit app
print("Starting Streamlit app...")

oauth2_callback_server_cmd = [sys.executable, "oauth2_callback_server.py", "--region", region]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    wait_for_oauth2_server_to_be_ready()

    # Run streamlit in the current directory
    process = subprocess.Popen([
        sys.executable, "-m", "streamlit", "run", "chatbot_app_cognito.py",
        "--server.port=8501", "--server.showEmailPrompt=false"
    ],
        cwd=notebook_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    # Print the output as it comes
    for line in iter(process.stdout.readline, ''):
        if line:
            if "8501" in line:
                print("\n🎉 Streamlit app is ready!")
                streamlit_url = get_streamlit_url()
                print(f'\n🚀 Streamlit Application URL:\n{streamlit_url}\n')
                print("⚠️ To stop the app, interrupt the kernel or press Ctrl+C in the terminal")
                break

except KeyboardInterrupt:
    print("\nStreamlit app stopped.")
    oauth2_callback_server_process.terminate()
    process.terminate()
except Exception as e:
    print(f"Error starting Streamlit app: {e}")

#### Option 2: Run from Terminal

Alternatively, you can run the Streamlit app from your terminal:

```bash
# Navigate to the current directory
cd 01-tutorials/03-AgentCore-identity/05-Outbound_Auth_3lo/

# Run the Streamlit app
streamlit run chatbot_app_cognito.py
```

#### Using the Streamlit App

1. **Login**: Use the credentials `testuser` / `MyPassword123!` (the default test user created by the Cognito setup)
2. **Chat**: Ask questions like "What is in my agenda for today?" or "Highlight main events from my agenda from today"
3. **OAuth Flow**: When the agent needs Google Calendar access, you'll see an authorization URL - click it to complete the OAuth flow
4. **Features**: The app includes:
   - Real-time streaming responses
   - Clickable URLs
   - Modern chat interface
   - Context awareness
   - Error handling with informative messages

The app automatically reads all configuration from your `.bedrock_agentcore.yaml` file, so it will use the same agent runtime you just deployed.

## Cleanup (Optional)

- Let's now clean up the AgentCore Runtime created
- Uncomment the below Cells and run.

In [ ]:
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
# agentcore_control_client = boto3.client(
#     'bedrock-agentcore-control',
#     region_name=region
# )
# ecr_client = boto3.client(
#     'ecr',
#     region_name=region
    
# )

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
    
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split('/')[1],
#     force=True
# )

## Congratulations!